In [1]:
import pandas as pd
import numpy as np
import spacy
import re
from spacy.tokenizer import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
#from torchtext.data.utils import get_tokenizer
#from torchtext.vocab import build_vocab_from_iterator
import tensorflow as tf
from datasets import load_dataset

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
from transformers import BertModel, BertConfig,BertTokenizer
from transformers.models.bert.modeling_bert import BertForPreTrainingOutput

In [3]:
configuration = BertConfig()
print(configuration)
model = BertModel(configuration)
print(model)
config = model.config    
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            

In [5]:

#dataset = load_dataset(
#   'imdb')


#dataset['train'].to_csv('imdb_train.csv')
#dataset['test'].to_csv('imdb_test.csv')

Reusing dataset imdb (C:\Users\Aris\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

32686819

In [4]:
dataset['train'][0]

NameError: name 'dataset' is not defined

In [5]:
df_train = pd.read_csv('imdb_train.csv')
df_test = pd.read_csv('imdb_test.csv')

print(df_train.head(10))
print(df_test.head(10))

   Unnamed: 0                                               text  label
0           0  I rented I AM CURIOUS-YELLOW from my video sto...      0
1           1  "I Am Curious: Yellow" is a risible and preten...      0
2           2  If only to avoid making this type of film in t...      0
3           3  This film was probably inspired by Godard's Ma...      0
4           4  Oh, brother...after hearing about this ridicul...      0
5           5  I would put this at the top of my list of film...      0
6           6  Whoever wrote the screenplay for this movie ob...      0
7           7  When I first saw a glimpse of this movie, I qu...      0
8           8  Who are these "They"- the actors? the filmmake...      0
9           9  This is said to be a personal film for Peter B...      0
   Unnamed: 0                                               text  label
0           0  I love sci-fi and am willing to put up with a ...      0
1           1  Worth the entertainment value of a rental, esp...

In [6]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [7]:
df_test['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [8]:
df_train

,Unnamed: 0,text,label
0,0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,2,If only to avoid making this type of film in t...,0
3,3,This film was probably inspired by Godard's Ma...,0
4,4,"Oh, brother...after hearing about this ridicul...",0
5,5,I would put this at the top of my list of film...,0
6,6,Whoever wrote the screenplay for this movie ob...,0
7,7,"When I first saw a glimpse of this movie, I qu...",0
8,8,"Who are these ""They""- the actors? the filmmake...",0
9,9,This is said to be a personal film for Peter B...,0


In [9]:
df_train['text'].to_csv("df_train.txt",index=False,header=True)


In [52]:
train_input = pd.read_csv("df_train.txt",delimiter="\t")
print(len(train_input))
print(train_input.iloc[0,:])

test_input = pd.read_csv("df_test.txt",delimiter="\t")
print(len(test_input))
print(test_input.iloc[0,:])


FileNotFoundError: [Errno 2] File b'df_train.txt' does not exist: b'df_train.txt'

In [33]:
#vocab = BertForPreTrainingOutput()
#print(vocab)
#token = BertTokenizer("df_train.txt")
#print(token)

BertForPreTrainingOutput(loss=None, prediction_logits=None, seq_relationship_logits=None, hidden_states=None, attentions=None)
PreTrainedTokenizer(name_or_path='', vocab_size=24905, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [28]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [50]:
Ids = []
Masks = []
for i in range(len(df_input)):
    print(df_input.iloc[i,:])
    inputs = tokenizer.encode_plus(str(df_input.iloc[i,:]),add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors="pt")
    Ids.append(inputs.get('input_ids'))
    Masks.append(inputs.get('attention_mask'))


text    I rented I AM CURIOUS-YELLOW from my video sto...
Name: 0, dtype: object
text    "I Am Curious: Yellow" is a risible and preten...
Name: 1, dtype: object
text    If only to avoid making this type of film in t...
Name: 2, dtype: object
text    This film was probably inspired by Godard's Ma...
Name: 3, dtype: object
text    Oh, brother...after hearing about this ridicul...
Name: 4, dtype: object
text    I would put this at the top of my list of film...
Name: 5, dtype: object
text    Whoever wrote the screenplay for this movie ob...
Name: 6, dtype: object
text    When I first saw a glimpse of this movie, I qu...
Name: 7, dtype: object
text    Who are these "They"- the actors? the filmmake...
Name: 8, dtype: object
text    This is said to be a personal film for Peter B...
Name: 9, dtype: object
text    It was great to see some of my favorite stars ...
Name: 10, dtype: object
text    I can't believe that those praising this movie...
Name: 11, dtype: object
text    Never cast models 

Name: 157, dtype: object
text    SPOILER: The young lover, Jed, is kicked out b...
Name: 158, dtype: object
text    The worst thing about Crush is not that it's a...
Name: 159, dtype: object
text    Not a `woman film' but film for the gang. One ...
Name: 160, dtype: object
text    As a single woman over 40, I found this film e...
Name: 161, dtype: object
text    I was gifted with this movie as it had such a ...
Name: 162, dtype: object
text    "The Crush" is a pleasant enough 40-something ...
Name: 163, dtype: object
text    it MIGHT have been a good movie if it had expl...
Name: 164, dtype: object
text    ***LIGHT SPOILER ALERT*** The story sounds goo...
Name: 165, dtype: object
text    This is a movie that relies solely on the some...
Name: 166, dtype: object
text    I am not so much like Love Sick as I image. Fi...
Name: 167, dtype: object
text    I am not so much like Love Sick as I image. Fi...
Name: 168, dtype: object
text    Considering that this movie had a serious and ...
Name

Name: 264, dtype: object
text    Blank check is one of those kids movies that c...
Name: 265, dtype: object
text    Blank Check is a movie that I saw on TV one da...
Name: 266, dtype: object
text    I saw this movie once a long time ago, and I h...
Name: 267, dtype: object
text    The worst movie I have seen in a while. Yeah i...
Name: 268, dtype: object
text    Blank Check is easily one of the worst films o...
Name: 269, dtype: object
text    I almost called HBO and demanded my money back...
Name: 270, dtype: object
text    This movie is really goofy! I saw it as an 11 ...
Name: 271, dtype: object
text    A young boy comes into a lot of money and prom...
Name: 272, dtype: object
text    Bad Actors, bad filming, choppy dialog, shallo...
Name: 273, dtype: object
text    Bad plot, bad dialogue, bad acting, idiotic di...
Name: 274, dtype: object
text    Larry Buchanan. Yep, same guy who did "Attack ...
Name: 275, dtype: object
text    what ends up killing this movie is its self-co...
Name

text    This is surely one of the worst films ever mad...
Name: 385, dtype: object
text    This film is about British prisoners of war fr...
Name: 386, dtype: object
text    Early film directed by D.W. Griffith; it featu...
Name: 387, dtype: object
text    In contrast to my fellow reviewers, I always t...
Name: 388, dtype: object
text    (aka: The Bloodsucker Leads the Dance)<br /><b...
Name: 389, dtype: object
text    Wow-this one sucks. I'm gonna sum it up as qui...
Name: 390, dtype: object
text    A particularly maligned example of Italian cul...
Name: 391, dtype: object
text    After an initial release of 4 very good Eurotr...
Name: 392, dtype: object
text    This movie is not only a very bad movie, with ...
Name: 393, dtype: object
text    La Sanguisuga Conduce la Danza, or The Bloodsu...
Name: 394, dtype: object
text    The Bloodsucker Leads the Dance - what a laugh...
Name: 395, dtype: object
text    The Horror Channel plays nothing but erotic so...
Name: 396, dtype: object
text

Name: 510, dtype: object
text    I don't give much credence to AIDS conspiracy ...
Name: 511, dtype: object
text    The cliché of the shell-shocked soldier home f...
Name: 512, dtype: object
text    The Good Earth is perhaps the most boring film...
Name: 513, dtype: object
text    The Good Earth is not a great film by any mean...
Name: 514, dtype: object
text    A couple of farmers struggle in life in a smal...
Name: 515, dtype: object
text    Luise Rainer received an Oscar for her perform...
Name: 516, dtype: object
text    Well I'm probably about to be lambasted by eve...
Name: 517, dtype: object
text    The Good Earth follows the life a slave girl a...
Name: 518, dtype: object
text    Well, what was fun... except for the fun part....
Name: 519, dtype: object
text    Amazing. That's what you'd say if you sat thro...
Name: 520, dtype: object
text    This movie could be used in film classes in a ...
Name: 521, dtype: object
text    I desperately want to give this movie a 10...I...
Name

...

Name: 24385, dtype: object
text    I am watching the series back to back as fast ...
Name: 24386, dtype: object
text    I am going to go out on a limb, and actually d...
Name: 24387, dtype: object
text    This is a very intriguing short movie by David...
Name: 24388, dtype: object
text    David Lynch's new short is a very "Lynchian" p...
Name: 24389, dtype: object
text    Hello, this little film is interesting especia...
Name: 24390, dtype: object
text    I really think that people are taking the wron...
Name: 24391, dtype: object
text    It's a short movie from David Lynch with just ...
Name: 24392, dtype: object
text    This is almost typical Lynch. However, What ma...
Name: 24393, dtype: object
text    I don't quite know how to explain "Darkend Roo...
Name: 24394, dtype: object
text    As has been well documented by previous poster...
Name: 24395, dtype: object
text    Rock Star: INXS was the best music TV series I...
Name: 24396, dtype: object
text    Don't know if this contains an

Name: 24493, dtype: object
text    They're not jawing journalists Cary Grant and ...
Name: 24494, dtype: object
text    As a baseball die-hard, this movie goes contra...
Name: 24495, dtype: object
text    I have always like this great baseball movie! ...
Name: 24496, dtype: object
text    I enjoy movies like this for their spirit, no ...
Name: 24497, dtype: object
text    That was definitely the case with Angels in th...
Name: 24498, dtype: object
text    I really enjoyed this movie as a young kid. At...
Name: 24499, dtype: object
text    Although I don't usually go for relentlessly h...
Name: 24500, dtype: object
text    At first you think another Disney movie, it mi...
Name: 24501, dtype: object
text    I think this is a lovely family movie. There a...
Name: 24502, dtype: object
text    I saw this film on TV many years ago and I saw...
Name: 24503, dtype: object
text    This was a top-notch movie with a top-notch ca...
Name: 24504, dtype: object
text    This film was pretty good. I a

text    I saw this film when I was 10 or 11 years old,...
Name: 24601, dtype: object
text    <br /><br />Film dominated by raven-haired Bar...
Name: 24602, dtype: object
text    Hardcastle and McCormick is an excellent TV sh...
Name: 24603, dtype: object
text    I took a chance on "Hardcastle and McCormick" ...
Name: 24604, dtype: object
text    Wonderful family drama/comedy starring MacClai...
Name: 24605, dtype: object
text    Shirley MacLaine in another tailor-made role. ...
Name: 24606, dtype: object
text    A toothsome little potboiler whose 65-minute l...
Name: 24607, dtype: object
text    Second-feature concerns a young woman in Londo...
Name: 24608, dtype: object
text    Having watched this film strictly on the stren...
Name: 24609, dtype: object
text    Nina Foch delivers a surprisingly strong perfo...
Name: 24610, dtype: object
text    Julia Ross (Nina Foch) agrees to take a positi...
Name: 24611, dtype: object
text    For those who think of Dame May Witty as the k...
Name: 2

text    As a collector of movie memorabilia, I had to ...
Name: 24715, dtype: object
text    Thanks to this fungal film I do NOT want my Ma...
Name: 24716, dtype: object
text    THE MELTING MAN...a tragic victim of the space...
Name: 24717, dtype: object
text    To me there is something so appealing and nost...
Name: 24718, dtype: object
text    I must admit I'm a little surprised and disapp...
Name: 24719, dtype: object
text    I searched out this one after seeing the hilar...
Name: 24720, dtype: object
text    This is the best made-for-TV movie of all-time...
Name: 24721, dtype: object
text    I have seen this movie and even though I kind ...
Name: 24722, dtype: object
text    i got to see the whole movie last night and i ...
Name: 24723, dtype: object
text    I saw True Crime when it was first released ba...
Name: 24724, dtype: object
text    i thought this movie was wonderfully plotted i...
Name: 24725, dtype: object
text    This is a nice piece of work. Very sexy and en...
Name: 2

Name: 24895, dtype: object
text    If there is a movie to be called perfect then ...
Name: 24896, dtype: object
text    A Chinese Ghost Story stars the late, great Le...
Name: 24897, dtype: object
text    Wasn't quite sure what to expect with this one...
Name: 24898, dtype: object
text    this is a great movie. i like it where ning cl...
Name: 24899, dtype: object
text    I can't remember many films where a bumbling i...
Name: 24900, dtype: object
text    Master director Ching Siu Tung's perhaps most ...
Name: 24901, dtype: object
text    It's sort of crazy, but I taped from TCM both,...
Name: 24902, dtype: object
text    This version of Anna Christie is in German. Gr...
Name: 24903, dtype: object
text    Filmed by MGM on the same sets as the English ...
Name: 24904, dtype: object
text    After Garbo's introduction to sound in Clarenc...
Name: 24905, dtype: object
text    Anna Christie (1931)<br /><br />On its own ter...
Name: 24906, dtype: object
text    Fragile Carne, just before his

NameError: name 'Print' is not defined